***Connect to google drive colab***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Installation of dependencies***

In [ ]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


***Display audio***

In [ ]:
from IPython.display import Audio

# Function to handle audio playback
def play_audio(file_path, rate=None):
    """Plays an audio file.

    Args:
        file_path (str): The path to the audio file.
        rate (int, optional): The sample rate of the audio.
                               If None, attempts to autodetect.
                               Defaults to None.
    """
    # If rate is not provided, assume it's an audio file and let Audio autodetect
    # If rate is provided, assume data is numpy array and pass rate
    audio = Audio(filename=file_path, rate=44100)
    display(audio)

audio_file = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals4.mp3"
play_audio(audio_file)

Output hidden; open in https://colab.research.google.com to view.

***Speech recognation***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals4.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the completed transcript object
result=transcript['text']
print(result)

Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive range but no immediate intervention needed. Monitoring trends for any significant elevations. Heart rate is steady at 70 beats per minute, the sinus rhythm noting a heart rate variability of 50 milliseconds which is within a healthy range indicating balanced autonomic function. Respiratory rate is measured at 18 breaths per minute euphenic with no signs of tachinia or bradyphenia moving on to oxygen saturation. SpO2 is recorded at 98 degrees 98% on room air indicative of sufficient oxygenation. No hypoxia detected. Core body temperature is measured at 98.5 degrees Fahrenheit via timefenic reading which falls within the normothermic range. Assessing end tidal CO2 levels patient has an ATCO 2 of 36mm Hg which is in the expected range showing effective ventilation. Cardiac output measured via non invasive monitor at 5.2 L per minute which is adequate for this patient's age and body w

In [ ]:
print(transcript)

{'id': '09464873-a87a-4879-a67c-67acdb2a1dee', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/3be27b3b-9fa0-4817-93a0-8ecfde30374a', 'text': "Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive range but no immediate intervention needed. Monitoring trends for any significant elevations. Heart rate is steady at 70 beats per minute, the sinus rhythm noting a heart rate variability of 50 milliseconds which is within a healthy range indicating balanced autonomic function. Respiratory rate is measured at 18 breaths per minute euphenic with no signs of tachinia or bradyphenia moving on to oxygen saturation. SpO2 is recorded at 98 degrees 98% on room air indicative of sufficient oxygenation. No hypoxia detected. Core body temperature is measured at 98.5 degrees Fahrenheit via timefenic reading which falls withi

'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397'

In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [transcript['text'][i:i+110] for i in range(0, len(transcript['text']), 110)]

# Join the chunks with line breaks
formatted_text = "\n".join(chunks)

print(formatted_text)

Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive ran
ge but no immediate intervention needed. Monitoring trends for any significant elevations. Heart rate is stead
y at 70 beats per minute, the sinus rhythm noting a heart rate variability of 50 milliseconds which is within 
a healthy range indicating balanced autonomic function. Respiratory rate is measured at 18 breaths per minute 
euphenic with no signs of tachinia or bradyphenia moving on to oxygen saturation. SpO2 is recorded at 98 degre
es 98% on room air indicative of sufficient oxygenation. No hypoxia detected. Core body temperature is measure
d at 98.5 degrees Fahrenheit via timefenic reading which falls within the normothermic range. Assessing end ti
dal CO2 levels patient has an ATCO 2 of 36mm Hg which is in the expected range showing effective ventilation. 
Cardiac output measured via non invasive monitor at 5.2 L per minute which is adequate for this patient's age 
a

***Speaker Diarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "speaker_labels": True
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals4.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcrSpeaker = create_transcript(your_api_token, upload_url)

# Iterate through the utterances and print speaker information
print("\nSpeakers' Information:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    print(f"Speaker {speaker}: {text}\nConfidence: {confidence}\n")


Speakers' Information:


Speaker A: Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive range but no immediate intervention needed. Monitoring trends for any significant elevations. Heart rate is steady at 70 beats per minute, the sinus rhythm noting a heart rate variability of 50 milliseconds which is within a healthy range indicating balanced autonomic function. Respiratory rate is measured at 18 breaths per minute euphenic with no signs of tachinia or bradyphenia moving on to oxygen saturation. SpO2 is recorded at 98 degrees 98% on room air indicative of sufficient oxygenation. No hypoxia detected. Core body temperature is measured at 98.5 degrees Fahrenheit via timefenic reading which falls within the normothermic range. Assessing end tidal CO2 levels patient has an ATCO 2 of 36mm Hg which is in the expected range showing effective ventilation. Cardiac output measured via non invasive monitor at 5.2 L per minute which is adequ

In [ ]:
print(transcrSpeaker['utterances'])

[{'speaker': 'A', 'text': "Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive range but no immediate intervention needed. Monitoring trends for any significant elevations. Heart rate is steady at 70 beats per minute, the sinus rhythm noting a heart rate variability of 50 milliseconds which is within a healthy range indicating balanced autonomic function. Respiratory rate is measured at 18 breaths per minute euphenic with no signs of tachinia or bradyphenia moving on to oxygen saturation. SpO2 is recorded at 98 degrees 98% on room air indicative of sufficient oxygenation. No hypoxia detected. Core body temperature is measured at 98.5 degrees Fahrenheit via timefenic reading which falls within the normothermic range. Assessing end tidal CO2 levels patient has an ATCO 2 of 36mm Hg which is in the expected range showing effective ventilation. Cardiac output measured via non invasive monitor at 5.2 L per minute which is adequate for th

***Summarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "summarization": True,
        "summary_model": "informative",
        "summary_type": "bullets"
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals4.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the summary
print(transcript["summary"])

- Patient vitals check for Mr. Anderson starting with blood pressure 12882 mm Hg within the pre hypertensive range but no immediate intervention needed. All vitals are stable with no immediate concern. Will continue regular monitoring and assess for any changes.


***Translate the text from english to any language the user want***

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='fr')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [result[i:i+chunk_size] for i in range(0, len(result), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



Vérification des signes vitaux du patient pour M. Anderson, commençant par une pression artérielle de 12 882 mm Hg dans la plage pré-hypertensive, mais aucune intervention immédiate n'est nécessaire. Surveillance des tendances pour toute élévation significative. La fréquence cardiaque est stable à 70 battements par minute, le rythme sinusal notant une variabilité de la fréquence cardiaque de 50 millisecondes, ce qui se situe dans une plage saine indiquant une fonction autonome équilibrée. La fréquence respiratoire est mesurée à 18 respirations par minute euphéniques sans aucun signe de tachinie ou de bradyphénie évoluant à la saturation en oxygène. La SpO2 est enregistrée à 98 degrés 98 % à l'air ambiant, ce qui indique une oxygénation suffisante. Aucune hypoxie détectée. La température corporelle centrale est mesurée à 98,5 degrés Fahrenheit via une lecture timefenic qui se situe dans la plage normothermique. En évaluant les niveaux de CO2 de fin d'expiration, le patient a un ATCO 2 d

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [transcript["summary"][i:i+chunk_size] for i in range(0, len(transcript["summary"]), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



- Die Vitalwerte des Patienten wurden bei Herrn Anderson überprüft. Der Blutdruck lag bei 12882 mmHg im prähypertensiven Bereich, aber es war kein sofortiger Eingriff erforderlich. Alle Vitalwerte sind stabil und es besteht kein unmittelbarer Grund zur Sorge. Die regelmäßige Überwachung wird fortgesetzt und auf etwaige Änderungen geprüft.


In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [translated_text[i:i+110] for i in range(0, len(translated_text), 110)]

# Join the chunks with line breaks
formatted_text_Trans= "\n".join(chunks)

print(formatted_text_Trans)

- Die Vitalwerte des Patienten wurden bei Herrn Anderson überprüft. Der Blutdruck lag bei 12882 mmHg im prähyp
ertensiven Bereich, aber es war kein sofortiger Eingriff erforderlich. Alle Vitalwerte sind stabil und es best
eht kein unmittelbarer Grund zur Sorge. Die regelmäßige Überwachung wird fortgesetzt und auf etwaige Änderunge
n geprüft.


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='mr')

# Iterate through the utterances and print translated speaker information
print("\nInformation des intervenants:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    # Translate the speaker and text
    translated_speaker = translator.translate(speaker)
    translated_text = translator.translate(text)

    print(f"Intervenant {translated_speaker}: {translated_text}\nConfiance: {confidence}\n")



Information des intervenants:


Intervenant ए: मिस्टर अँडरसनसाठी रुग्णाच्या जीवनावश्यक गोष्टी तपासल्या जातात ज्याचा रक्तदाब 12882 मिमी एचजी पासून उच्च रक्तदाबपूर्व श्रेणीमध्ये सुरू होतो परंतु त्वरित हस्तक्षेप आवश्यक नाही. कोणत्याही महत्त्वाच्या उंचीसाठी ट्रेंडचे निरीक्षण करणे. हृदय गती 70 बीट्स प्रति मिनिटावर स्थिर आहे, सायनस ताल 50 मिलीसेकंदांच्या हृदय गतीची परिवर्तनशीलता लक्षात घेते जी संतुलित स्वायत्त कार्य दर्शविणारी निरोगी श्रेणीमध्ये असते. ऑक्सिजन संपृक्ततेकडे टॅचिनिया किंवा ब्रॅडीफेनियाची कोणतीही चिन्हे नसताना श्वसन दर 18 श्वासोच्छ्वास प्रति मिनिट युफेनिक मोजला जातो. पुरेशा ऑक्सिजनचे सूचक खोलीतील हवेवर SpO2 98 अंश 98% नोंदवले जाते. हायपोक्सिया आढळला नाही. मुख्य शरीराचे तापमान टाइमफेनिक रीडिंगद्वारे 98.5 अंश फॅरेनहाइटवर मोजले जाते जे नॉर्मोथर्मिक श्रेणीमध्ये येते. शेवटच्या भरतीच्या CO2 पातळीचे मूल्यांकन करताना रुग्णाला 36mm Hg चा ATCO 2 असतो जो प्रभावी वायुवीजन दर्शविणारी अपेक्षित श्रेणीत असतो. ह्रदयाचा आउटपुट नॉन-इनवेसिव्ह मॉनिटरद्वारे 5.2 एल प्रति मिनिटाने मोजला जातो जो या रुग